<img src="https://raw.githubusercontent.com/EO-College/cubes-and-clouds/main/icons/cnc_3icons_process_circle.svg"
     alt="Cubes & Clouds logo"
     style="float: center; margin-right: 10px;" />

# 2.3 Data Access and Basic Processing

## Resample Operators

Sometimes we need to align the spatial or temporal dimension of two datacubes, so that they can be merged together.

### `resample_cube_spatial`: spatial resampling Sentinel-2 to match Landsat

Start importing the necessary libraries and initialize a local connection for Client-Side Processing.

In [3]:
import openeo
from openeo.local import LocalConnection
local_conn = LocalConnection('')

Create two datacubes, one for Sentinel-2 and one for Landsat

In [8]:
url = "https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a"

spatial_extent = {"west": 11.4, "east": 11.42, "south": 45.5, "north": 45.52}
temporal_extent = ["2023-06-01", "2023-06-30"]
bands = ["red","nir"]

s2_cube = local_conn.load_stac(url=url,
   spatial_extent=spatial_extent,
   temporal_extent=temporal_extent,
   bands=bands
)
s2_cube.execute()

<xarray.DataArray 'stackstac-bb50aa855b81341fb27483e2305bf5e0' (time: 6,
                                                                band: 2,
                                                                y: 227, x: 163)>
dask.array<getitem, shape=(6, 2, 227, 163), dtype=float64, chunksize=(1, 1, 227, 163), chunktype=numpy.ndarray>
Coordinates: (12/54)
  * time                                     (time) datetime64[ns] 2023-06-02...
    id                                       (time) <U24 'S2A_32TPR_20230602_...
  * band                                     (band) <U3 'red' 'nir'
  * x                                        (x) float64 6.874e+05 ... 6.891e+05
  * y                                        (y) float64 5.044e+06 ... 5.041e+06
    s2:medium_proba_clouds_percentage        (time) float64 16.89 ... 18.45
    ...                                       ...
    proj:shape                               object {10980}
    title                                    (band) <U20 'Red (band 4) - 10m'...
    common_name                              (band) <U3 'red' 'nir'
    center_wavelength                        (band) float64 0.665 0.842
    full_width_half_max                      (band) float64 0.038 0.145
    epsg                                     int32 32632
Attributes:
    spec:        RasterSpec(epsg=32632, bounds=(600000.0, 4990200.0, 709800.0...
    crs:         epsg:32632
    transform:   | 10.00, 0.00, 600000.00|\n| 0.00,-10.00, 5100000.00|\n| 0.0...
    resolution:  10.0

In [9]:
url = "https://planetarycomputer.microsoft.com/api/stac/v1/collections/landsat-c2-l2"
bands = ["red","nir08"]
l8_cube = local_conn.load_stac(url=url,
                    spatial_extent=spatial_extent,
                    temporal_extent=temporal_extent,
                    bands=bands)
l8_cube.execute()

<xarray.DataArray 'stackstac-dd7bdb708bdf6a8e5c096fa08e5c1385' (time: 15,
                                                                band: 2, y: 75,
                                                                x: 54)>
dask.array<getitem, shape=(15, 2, 75, 54), dtype=float64, chunksize=(1, 1, 75, 54), chunktype=numpy.ndarray>
Coordinates: (12/27)
  * time                         (time) datetime64[ns] 2023-06-03T10:03:52.60...
    id                           (time) <U31 'LC08_L2SP_193028_20230603_02_T1...
  * band                         (band) <U5 'red' 'nir08'
  * x                            (x) float64 6.874e+05 6.875e+05 ... 6.89e+05
  * y                            (y) float64 5.044e+06 5.044e+06 ... 5.041e+06
    landsat:cloud_cover_land     (time) float64 74.1 91.94 74.7 ... 73.45 73.56
    ...                           ...
    landsat:wrs_row              (time) <U3 '028' '028' '029' ... '028' '029'
    sci:doi                      (time) <U16 '10.5066/P9OGBGM6' ... '10.5066/...
    landsat:scene_id             (time) <U21 'LC81930282023154LGN00' ... 'LC8...
    raster:bands                 object {'scale': 2.75e-05, 'nodata': 0, 'off...
    title                        (band) <U22 'Red Band' 'Near Infrared Band 0.8'
    epsg                         int32 32632
Attributes:
    spec:        RasterSpec(epsg=32632, bounds=(495570.0, 4825470.0, 855630.0...
    crs:         epsg:32632
    transform:   | 30.00, 0.00, 495570.00|\n| 0.00,-30.00, 5217630.00|\n| 0.0...
    resolution:  30.0

From the previous outputs, notice the shape difference in the spatial dimensions `x` and `y`.

This is due to the different resolution of the two collections: 10m for Sentinel-2, 30m for Landsat.

Now we use the `resample_cube_spatial` process to resample the Sentinel-2 data to match Landsat.

In [7]:
s2_cube_30m = s2_cube.resample_cube_spatial(target=l8_cube,method="average")
s2_cube_30m

Check what happens to the datacube inspecting the resulting xArray object. Now the `x` and `y` shape is the same as Landsat:

In [10]:
s2_cube_30m.execute()

<xarray.DataArray 'reproject-cf12447e0bf14db5987b7c3e606813ed' (time: 6,
                                                                band: 2, y: 75,
                                                                x: 54)>
dask.array<transpose, shape=(6, 2, 75, 54), dtype=float64, chunksize=(1, 1, 75, 54), chunktype=numpy.ndarray>
Coordinates: (12/55)
  * time                                     (time) datetime64[ns] 2023-06-02...
    id                                       (time) <U24 'S2A_32TPR_20230602_...
  * band                                     (band) <U3 'red' 'nir'
    s2:medium_proba_clouds_percentage        (time) float64 16.89 ... 18.45
    constellation                            <U10 'sentinel-2'
    s2:not_vegetated_percentage              (time) float64 15.78 ... 8.217
    ...                                       ...
    center_wavelength                        (band) float64 0.665 0.842
    full_width_half_max                      (band) float64 0.038 0.145
    epsg                                     int32 32632
  * y                                        (y) float64 5.044e+06 ... 5.041e+06
  * x                                        (x) float64 6.874e+05 ... 6.89e+05
    spatial_ref                              int32 0
Attributes:
    spec:        RasterSpec(epsg=32632, bounds=(600000.0, 4990200.0, 709800.0...
    transform:   | 10.00, 0.00, 600000.00|\n| 0.00,-10.00, 5100000.00|\n| 0.0...
    resolution:  10.0

### `resample_cube_temporal`: temporal resampling Sentinel-2 to match Landsat

In [15]:
s2_cube.resample_cube_temporal(target=l8_cube)